In [20]:
import pandas as pd
import requests
import io
import os
from dotenv import load_dotenv

In [21]:
load_dotenv()

True

In [62]:
url = "https://docs.dagster.io/assets/iris.csv"
s = requests.get(url).content
c = pd.read_csv(
    io.StringIO(s.decode("utf-8")),
    names=[
        "sepal_length_cm",
        "sepal_width_cm",
        "petal_length_cm",
        "petal_width_cm",
        "species",
    ],
)

In [63]:
c.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


bigquery

In [18]:
import google.auth

credentials, project = google.auth.default()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [56]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)

In [57]:
from google.cloud import bigquery

client = bigquery.Client(credentials=credentials)

In [39]:
datasets = list(client.list_datasets())

In [42]:
for dataset in datasets:
    print(dataset.dataset_id)

bt_syn_individual_test
dbt_emason
elh_pdg_elmo
iris
jaffle_shop
pokemon_api
stripe


In [58]:
tables = list(client.list_tables("iris"))

In [60]:
for table in tables:
    print(table.table_id)

iris_data


dataframe.to_gbq

In [111]:
c.to_gbq("iris.iris_data", credentials=credentials, if_exists="append")

100%|██████████| 1/1 [00:00<00:00, 1695.35it/s]


duckdb testing

In [65]:
import duckdb

duck_conn = duckdb.connect()

In [67]:
duck_conn.sql("select * from 'https://pokeapi.co/api/v2/pokemon'").fetchall()

CatalogException: Catalog Error: Table with name https://pokeapi.co/api/v2/pokemon/ditto does not exist!
Did you mean "pg_depend"?

In [77]:
response = requests.get("https://pokeapi.co/api/v2/pokemon?limit=2000")

In [78]:
response.json()

{'count': 1302,
 'next': None,
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
  {

In [81]:
df_pokemon = pd.json_normalize(response.json()["results"])

In [85]:
response_type = requests.get("https://pokeapi.co/api/v2/type")

In [89]:
df_types = pd.DataFrame(response_type.json()["results"])

In [93]:
response = requests.get(f"https://pokeapi.co/api/v2/type/{df_types.iloc[0,0]}")

In [97]:
pd.json_normalize(response.json()["pokemon"])

,slot,pokemon.name,pokemon.url
0,1,pidgey,https://pokeapi.co/api/v2/pokemon/16/
1,1,pidgeotto,https://pokeapi.co/api/v2/pokemon/17/
2,1,pidgeot,https://pokeapi.co/api/v2/pokemon/18/
3,1,rattata,https://pokeapi.co/api/v2/pokemon/19/
4,1,raticate,https://pokeapi.co/api/v2/pokemon/20/
...,...,...,...
153,1,squawkabilly-yellow-plumage,https://pokeapi.co/api/v2/pokemon/10261/
154,1,squawkabilly-white-plumage,https://pokeapi.co/api/v2/pokemon/10262/
155,2,ursaluna-bloodmoon,https://pokeapi.co/api/v2/pokemon/10272/
156,1,terapagos-terastal,https://pokeapi.co/api/v2/pokemon/10276/


In [103]:
df_append = pd.DataFrame()
for pokemon_type in df_types["name"]:
    print(pokemon_type)
    response = requests.get(f"https://pokeapi.co/api/v2/type/{pokemon_type}")
    df_temp = pd.json_normalize(response.json()["pokemon"])
    df_temp["type"] = pokemon_type
    df_append = pd.concat([df_append, df_temp], axis=0)

normal
fighting
flying
poison
ground
rock
bug
ghost
steel
fire
water
grass
electric
psychic
ice
dragon
dark
fairy
unknown
shadow


In [110]:
df_append.loc[df_append["pokemon.name"].str.contains("ratt")]

,slot,pokemon.name,pokemon.url,type
3,1.0,rattata,https://pokeapi.co/api/v2/pokemon/19/,normal
136,2.0,rattata-alola,https://pokeapi.co/api/v2/pokemon/10091/,normal
76,1.0,rattata-alola,https://pokeapi.co/api/v2/pokemon/10091/,dark


In [112]:
response = requests.get("https://pokeapi.co/api/v2/region/")

In [113]:
response.status_code

200

In [114]:
response.json()

{'count': 10,
 'next': None,
 'previous': None,
 'results': [{'name': 'kanto', 'url': 'https://pokeapi.co/api/v2/region/1/'},
  {'name': 'johto', 'url': 'https://pokeapi.co/api/v2/region/2/'},
  {'name': 'hoenn', 'url': 'https://pokeapi.co/api/v2/region/3/'},
  {'name': 'sinnoh', 'url': 'https://pokeapi.co/api/v2/region/4/'},
  {'name': 'unova', 'url': 'https://pokeapi.co/api/v2/region/5/'},
  {'name': 'kalos', 'url': 'https://pokeapi.co/api/v2/region/6/'},
  {'name': 'alola', 'url': 'https://pokeapi.co/api/v2/region/7/'},
  {'name': 'galar', 'url': 'https://pokeapi.co/api/v2/region/8/'},
  {'name': 'hisui', 'url': 'https://pokeapi.co/api/v2/region/9/'},
  {'name': 'paldea', 'url': 'https://pokeapi.co/api/v2/region/10/'}]}

In [115]:
response = requests.get("https://pokeapi.co/api/v2/region/johto")

In [121]:
pd.json_normalize(response.json())

,id,locations,name,names,pokedexes,version_groups,main_generation.name,main_generation.url
0,2,"[{'name': 'blackthorn-city', 'url': 'https://p...",johto,"[{'language': {'name': 'ja-Hrkt', 'url': 'http...","[{'name': 'original-johto', 'url': 'https://po...","[{'name': 'gold-silver', 'url': 'https://pokea...",generation-ii,https://pokeapi.co/api/v2/generation/2/


In [123]:
response = requests.get("https://pokeapi.co/api/v2/pokemon-species")

In [124]:
response.status_code

200

In [125]:
response.json()

{'count': 1025,
 'next': 'https://pokeapi.co/api/v2/pokemon-species?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon-species/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon-species/3/'},
  {'name': 'charmander',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/4/'},
  {'name': 'charmeleon',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon-species/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon-species/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon-species/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon-species/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon-species/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon-species/11

In [130]:
from dagster import file_relative_path

duckdb_database_test = file_relative_path(__file__, "../data/db/osmds.db")

NameError: name '__file__' is not defined

In [1]:
import duckdb

In [16]:
duck_conn = duckdb.connect("../../data/db/pokemon.duckdb")

In [12]:
duck_conn.sql("show all tables").fetchall()

[('pokemon',
  'main',
  'pokemon',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False),
 ('pokemon',
  'main',
  'pokemon_types',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False)]

In [13]:
duck_conn.execute("select * from pokemon_types limit 10;").fetchall()

[('normal', 'https://pokeapi.co/api/v2/type/1/'),
 ('fighting', 'https://pokeapi.co/api/v2/type/2/'),
 ('flying', 'https://pokeapi.co/api/v2/type/3/'),
 ('poison', 'https://pokeapi.co/api/v2/type/4/'),
 ('ground', 'https://pokeapi.co/api/v2/type/5/'),
 ('rock', 'https://pokeapi.co/api/v2/type/6/'),
 ('bug', 'https://pokeapi.co/api/v2/type/7/'),
 ('ghost', 'https://pokeapi.co/api/v2/type/8/'),
 ('steel', 'https://pokeapi.co/api/v2/type/9/'),
 ('fire', 'https://pokeapi.co/api/v2/type/10/')]

In [14]:
duck_conn.execute("select * from pokemon limit 10;").fetchall()

[('bulbasaur', 'https://pokeapi.co/api/v2/pokemon/1/'),
 ('ivysaur', 'https://pokeapi.co/api/v2/pokemon/2/'),
 ('venusaur', 'https://pokeapi.co/api/v2/pokemon/3/'),
 ('charmander', 'https://pokeapi.co/api/v2/pokemon/4/'),
 ('charmeleon', 'https://pokeapi.co/api/v2/pokemon/5/'),
 ('charizard', 'https://pokeapi.co/api/v2/pokemon/6/'),
 ('squirtle', 'https://pokeapi.co/api/v2/pokemon/7/'),
 ('wartortle', 'https://pokeapi.co/api/v2/pokemon/8/'),
 ('blastoise', 'https://pokeapi.co/api/v2/pokemon/9/'),
 ('caterpie', 'https://pokeapi.co/api/v2/pokemon/10/')]

In [15]:
duck_conn.close()

# example to build tables using previous endpoint responses as iterables

In [36]:
# pokemon_response = requests.get("https://pokeapi.co/api/v2/pokemon?limit=2000")
df_pokemon_append = pd.DataFrame()

for pokemon in duck_conn.execute("select name from pokemon limit 5;").fetchall():
    pokemon_response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon[0]}")

    temp_df = pd.json_normalize(pokemon_response.json())

    df_pokemon_append = pd.concat([df_pokemon_append, temp_df], axis=0)

In [39]:
len(df_pokemon_append.columns)

139

columns of interest:
abilities, base_experience, forms
game_indicies, height, held_items (Boolean?)
id, moves, 
stats, types, weight

In [101]:
# how to switch row values from a column to column indices

pd.json_normalize(df_pokemon_append.iloc[1:2, 14:15].values[0][0]).pivot_table(
    "base_stat", None, "stat.name"
)

stat.name,attack,defense,hp,special-attack,special-defense,speed
base_stat,62,63,60,80,80,60


In [25]:
pokemon_response = requests.get("https://pokeapi.co/api/v2/pokemon/bulbasaur")

In [27]:
pokemon_response.json()

{'abilities': [{'ability': {'name': 'overgrow',
    'url': 'https://pokeapi.co/api/v2/ability/65/'},
   'is_hidden': False,
   'slot': 1},
  {'ability': {'name': 'chlorophyll',
    'url': 'https://pokeapi.co/api/v2/ability/34/'},
   'is_hidden': True,
   'slot': 3}],
 'base_experience': 64,
 'forms': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}],
 'game_indices': [{'game_index': 153,
   'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}},
  {'game_index': 153,
   'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}},
  {'game_index': 153,
   'version': {'name': 'yellow',
    'url': 'https://pokeapi.co/api/v2/version/3/'}},
  {'game_index': 1,
   'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version/4/'}},
  {'game_index': 1,
   'version': {'name': 'silver',
    'url': 'https://pokeapi.co/api/v2/version/5/'}},
  {'game_index': 1,
   'version': {'name': 'crystal',
    'url': 'https://pokeapi.co/

In [40]:
duck_conn.close()